In [ ]:
from pydub import AudioSegment
from pydub.silence import detect_nonsilent, detect_silence
from pydub.silence import split_on_silence
import numpy as np
import ffmpeg

from os import listdir
from os.path import isfile, join

#adjust target amplitude
def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)

In [ ]:
with open("./records/_records.txt") as f:
    contents = f.readlines()
onlyfiles = [f for f in listdir("./records/") if isfile(join("./records/", f))]
onlyfiles = list(filter(lambda x: ".wav" in x, onlyfiles))

In [ ]:
data = []
for file in onlyfiles:
#Convert wav to audio_segment
    name = file[:file.rfind(".")]
    audio_segment = AudioSegment.from_wav("./records/" + name + ".wav")

    #normalize audio_segment to -20dBFS 
    normalized_sound = match_target_amplitude(audio_segment, -20.0)
    print("length of audio_segment = {} seconds".format(len(normalized_sound)/1000))

    #Print detected non-silent chunks, which in our case would be spoken words.
    nonsilent_data = detect_silence(normalized_sound, min_silence_len=500, silence_thresh=-25, seek_step=1)

    print("start, stop")
    flattened = []
    combined = []
    for tuple in nonsilent_data:
        for value in tuple:
            flattened.append(value)

    flattened = np.array(flattened)

    res = list(filter(lambda x: name in x, contents))[0]
    mood = 0
    if "bad" in res:
        mood = 0
    if "neutral" in res:
        mood = 1
    if "good" in res:
        mood = 2
    flattened_pmeta = [mood, flattened]
    flattened_pmeta = np.array(flattened_pmeta)
    data.append(flattened_pmeta)

data = np.array(data)

In [ ]:
import pandas as pd

# Convert `X` to data frame
df = pd.DataFrame(data)

# Rename columns
df.columns = ['feature_' + str(i + 1) for i in range(df.shape[1])]

# Convert the feature with lists inside to long format
features = df['feature_2'].explode().to_frame()

# Create counter by observation so we can pivot
features['observation_id'] = features.groupby(level=0).cumcount()

# Convert to dataset and rename all columns
features = features.pivot(columns='observation_id', values='feature_2').fillna(0)
features = features.add_prefix('list_element_')

# Drop `feature_5` from X
df.drop(columns='feature_2', axis=1, inplace=True)

# Concatenate X and x together
df = pd.concat([df, features], axis=1)

In [ ]:
df.head(5)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop(["feature_1"],axis=1)
y = df["feature_1"]
y=y.astype('int')

In [ ]:
# Carry on as before
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 44)

In [ ]:
from sklearn.svm import SVC
model  = SVC()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
score = model.score(X_test, y_test)
score

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model  = GradientBoostingClassifier()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
score = model.score(X_test, y_test)
score